In [3]:
import pandas as pd
from jh_utils.data.sql.object import create_object_DB_by_envfile
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [48]:
import sys
sys.path.insert(1, '../../')

In [49]:
from src.d03_model_evaluation.evaluate import evaluate

ModuleNotFoundError: No module named 'sklearn'

In [34]:
def make_dummies(series):
    df = pd.get_dummies(series,drop_first=True)
    df.columns = list(map(lambda x: '{}_'.format(series.name) + str(x),list(df.columns)))
    return df

In [35]:
db = create_object_DB_by_envfile('../../.env')

In [36]:
db

host: localhost
db:weather

In [37]:
query = 'select * from datasets."ES_0"'
db.connect()
df = pd.read_sql(query,con=db.engine)

In [38]:
df.date_time = pd.to_datetime(df.date_time)
df['hour'] = df.date_time.dt.hour
df['month'] = df.date_time.dt.month
df['year'] = df.date_time.dt.year
df['day_of_year'] = df.date_time.dt.day_of_year
df['weekofyear'] = df.date_time.dt.weekofyear

## transforming start hour in 9, to use hour**3, some models are hierarchical so is necessary to keep hour**2 
df['hour_9'] = df['hour'].apply(lambda x: (x-9)%24)
df['hour_9**2'] = df['hour_9']**2
df['hour_9**3'] = df['hour_9']**3

/tmp/ipykernel_10442/2037374672.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df.date_time.dt.weekofyear


In [42]:
df_model = df[['date_time','A612 - temperatura maxima na hora ant. (aut) (°c)','month','hour_9', 'hour_9**2', 'hour_9**3']]

In [43]:
df_month = make_dummies(df['month'])
df_model = pd.concat([df_model,df_month],axis=1)

In [45]:
df_model.drop(['month'], axis=1,inplace=True)

In [46]:
df_model

,date_time,A612 - temperatura maxima na hora ant. (aut) (°c),hour_9,hour_9**2,hour_9**3,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2006-11-01 00:00:00,24.9,15,225,3375,0,0,0,0,0,0,0,0,0,1,0
1,2006-11-01 01:00:00,24.8,16,256,4096,0,0,0,0,0,0,0,0,0,1,0
2,2006-11-01 02:00:00,24.9,17,289,4913,0,0,0,0,0,0,0,0,0,1,0
3,2006-11-01 03:00:00,24.6,18,324,5832,0,0,0,0,0,0,0,0,0,1,0
4,2006-11-01 04:00:00,24.4,19,361,6859,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127075,2021-04-30 19:00:00,25.9,10,100,1000,0,0,1,0,0,0,0,0,0,0,0
127076,2021-04-30 20:00:00,24.8,11,121,1331,0,0,1,0,0,0,0,0,0,0,0
127077,2021-04-30 21:00:00,23.7,12,144,1728,0,0,1,0,0,0,0,0,0,0,0
127078,2021-04-30 22:00:00,23.1,13,169,2197,0,0,1,0,0,0,0,0,0,0,0


In [ ]:

def ols_model(x, y, codename=None):
    if codename == None:
        codename = f'{x.shape[1]}_{y.name.split("_")[1]}'
        
    x_train, x_test = x[df_model.date_time<'01-01-2018'], x[df_model.date_time>='01-01-2018']
    y_train, y_test = y[df_model.date_time<'01-01-2018'], y[df_model.date_time>='01-01-2018']
    
    model = OLS(y_train,x_train)
    fit = model.fit()
    y_pred = fit.predict(x_test)
    
    metrics = evaluate(y_test,y_pred)
    return {'y_pred': y_pred, 'fit': fit, 'model':model, 'metrics':metrics, 'codename': codename}

model = ols_model(x,_y_.iloc[:,1])
